In [ ]:
!pip install sentence_transformers
!pip install chromadb
!pip install --upgrade "pymongo"
!pip install feedparser newspaper3k nltk 
!pip install lxml_html_clean
!pip -q install pypdf tldextract
import requests
import os
import csv
import json
import pandas as pd
import feedparser
import chromadb
import nltk
nltk.download('punkt')
# import nltk
nltk.download('punkt_tab')
import time
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from chromadb import Client, Settings
from pymongo import MongoClient
from newspaper import Article
from nltk.tokenize import sent_tokenize
import hashlib
from xml.etree import ElementTree as ET
from urllib.parse import urljoin
from urllib.parse import urlparse

model = SentenceTransformer('sentence-transformers/LaBSE')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
team = "patriots"
hub_url = "https://www.patriots.com/news/transcripts"
    

In [3]:
# fetching the patriots team hub page for interviews using BeautifulSoup
HEADERS = {"User-Agent": "Mozilla/5.0 (CapstoneResearchBot/1.0)"}
resp = requests.get(hub_url, headers=HEADERS, timeout = 20)
print("status", resp.status_code)
print("content type", resp.headers.get("content type"))
print("first characters ", resp.text[:500])

status 200
content type None
first characters  

<!DOCTYPE html>
    <html lang="en-US" dir="ltr" />

<head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />

  <link rel="dns-prefetch" href="//nflenterprises.tt.omtrdc.net" crossorigin>

  <link rel="preconnect" href="//securepubads.g.doubleclick.net" crossorigin>
    <link rel="preconnect" href="//cdn.onesignal.com" crossorigin>
    <link rel="preconnect" href="//ass


In [4]:
soup = BeautifulSoup(resp.text, "html.parser")
links = soup.find_all("a",href=True)
print("total links", len(links))

total links 164


In [5]:
for a in links[:30]:
    print(a["href"], "||", a.get_text(" ", strip=True)[:80])

#main-content || Skip to main content
https://www.patriots.com || 
https://www.patriots.com || 
https://www.patriots.com/news/index || NEWS
https://www.patriots.com/video/index || VIDEO
https://www.patriots.com/audio/podcasts || PODCASTS
https://www.patriots.com/photos/index || PHOTOS
https://www.patriots.com/schedule/index || SCHEDULE
https://www.patriots.com/team/players-roster/index || TEAM
https://www.patriots.com/community/ || COMMUNITY
#2ndlevel || 
https://www.patriots.com/app || MOBILE APP
https://www.patriots.com/alumni || ALUMNI
https://www.patriots.com/cheerleaders/index || CHEERLEADERS
https://www.patriots.com/trainingcamp || TRAINING CAMP
https://www.thehallatpatriotplace.com/ || HALL OF FAME
https://www.patriots.com/fans/index || FAN ZONE
https://www.patriots.com/press-room/index || PRESS ROOM
https://proshop.patriots.com/?_s=bm-teamsiteshopbutton-Patriots || PROSHOP
https://www.patriots.com/tickets/index || TICKETS
https://proshop.patriots.com/?_s=bm-teamsiteshopbutton-P

In [9]:
from urllib.parse import urljoin, urlparse
hub_domain = urlparse(hub_url).netloc
rows = []
seen = set()
for a in links:
    href = a["href"].strip()
    abs_url = urljoin(hub_url, href)

    if urlparse(abs_url).netloc != hub_domain:
        continue
    anchor_text = a.get_text(" ", strip=True)

    if not re.search(r"(transcript|media-availability|postgame|press-conference|quotes)", abs_url, re.I):
        if not re.search(r"(Transcript|Postgame|Media Availability|Press Conference|Quotes)", anchor_text, re.I):
            continue
links


[<a class="d3-u-block-bypass d3-u-block-bypass--with-size" href="#main-content" tabindex="0">
 <span>Skip to main content</span>
 </a>,
 <a data-link_module="Branded Header" data-link_name="" data-link_sitename="" data-link_type="_header" data-link_url="https://www.patriots.com" href="https://www.patriots.com">
 <picture is-lazy="/t_lazy">
 <!--[if IE 9]><video style="display: none;"><![endif]-->
 <source data-srcset="https://static.clubs.nfl.com/image/upload/patriots/qnbhkiysuj04iv4rmqi1.png" media="(min-width: 1024px)"/>
 <source data-srcset="https://static.clubs.nfl.com/image/upload/patriots/o3aj1wkezymz3cibeuxn.png" media="(min-width: 320px)"/>
 <!--[if IE 9]></video><![endif]-->
 
 </picture>
 </a>,
 <a class="d3-o-nav__logo" data-event_name="click action" data-link_module="Header" data-link_name="Nav Logo" data-link_type="Nav Logo" data-link_url="/" href="https://www.patriots.com" title="Link to club's homepage">
 <picture><!--[if IE 9]><video style="display: none; "><![endif]-->

In [16]:
hits = []
looking_for = re.compile(r"(transcript|media availability|media-availability|postgame|press[- ]conference|quotes)", re.I)
for a in links:
    href = a.get("href", "")
    text = a.get_text(" ",strip=True)
    if looking_for.search(href) or looking_for.search(text):
        hits.append((href, text[:120]))
print("hit count:", len(hits))
for h in hits[:30]:
    print(h)

hit count: 25
('https://www.patriots.com/news/transcripts', 'Transcripts')
('/news/transcripts-head-coach-mike-vrabel-quarterback-drake-maye-and-wide-receiver-stefon-diggs-press-conferences-2-4', 'Transcripts: Head Coach Mike Vrabel, Quarterback Drake Maye and Wide Receiver Stefon Diggs Press Conferences 2/4 Read th')
('/news/transcript-head-coach-mike-vrabel-press-conference-1-30', "Transcript: Head Coach Mike Vrabel Press Conference 1/30 Read the full transcript from Head Coach Mike Vrabel's press co")
('/news/transcript-head-coach-mike-vrabel-press-conference-1-29', "Transcript: Head Coach Mike Vrabel Press Conference 1/29 Read the full transcript from Head Coach Mike Vrabel's press co")
('/news/transcript-drake-maye-press-conference-1-29', "Transcript: Drake Maye Press Conference 1/29 Read the full transcript from Drake Maye's press conference on Thursday, Ja")
('/news/transcript-head-coach-mike-vrabel-press-conference-1-27', "Transcript: Head Coach Mike Vrabel Press Conference 1/2

In [18]:
base_link = "https://www.patriots.com"
hits_urls = [(urljoin(base_link, href), text) 
             for href, text in hits]
hits_urls[:5]

[('https://www.patriots.com/news/transcripts', 'Transcripts'),
 ('https://www.patriots.com/news/transcripts-head-coach-mike-vrabel-quarterback-drake-maye-and-wide-receiver-stefon-diggs-press-conferences-2-4',
  'Transcripts: Head Coach Mike Vrabel, Quarterback Drake Maye and Wide Receiver Stefon Diggs Press Conferences 2/4 Read th'),
 ('https://www.patriots.com/news/transcript-head-coach-mike-vrabel-press-conference-1-30',
  "Transcript: Head Coach Mike Vrabel Press Conference 1/30 Read the full transcript from Head Coach Mike Vrabel's press co"),
 ('https://www.patriots.com/news/transcript-head-coach-mike-vrabel-press-conference-1-29',
  "Transcript: Head Coach Mike Vrabel Press Conference 1/29 Read the full transcript from Head Coach Mike Vrabel's press co"),
 ('https://www.patriots.com/news/transcript-drake-maye-press-conference-1-29',
  "Transcript: Drake Maye Press Conference 1/29 Read the full transcript from Drake Maye's press conference on Thursday, Ja")]

In [21]:
# taking the links and organizing them into three categories: 
    # player press conferences, post-game quotes and then everything else
# start with some helper functions to organize
def is_coach_item(anchor_text: str) -> bool:
    t = (anchor_text or "").lower()
    return("coach" in t) or ("head coach" in t)
def is_press_conf(anchor_text: str) -> bool:
    t = t = (anchor_text or "").lower()
    return("transcript" in t) and ("press conference" in t) and (not is_coach_item(t))
def is_postgame_quotes(anchor_text: str) -> bool:
    t = (anchor_text or "").lower()
    return("postgame quotes" in t)
# put each item in their respective list
player_press_conference = []
other = []
postgame_quotes = []

for url, text in hits_urls:
    if is_press_conf(text):
        player_press_conference.append((url,text))
    elif is_postgame_quotes(text):
        postgame_quotes.append((url,text))
    else:
        other.append((url,text))
print("press conferences:", len(player_press_conference))
print("postgame:", len(postgame_quotes))
print("others:", len(other))




press conferences: 6
postgame: 4
others: 15


In [22]:
player_press_conference[0]

('https://www.patriots.com/news/transcript-drake-maye-press-conference-1-29',
 "Transcript: Drake Maye Press Conference 1/29 Read the full transcript from Drake Maye's press conference on Thursday, Ja")

In [23]:
test_url = player_press_conference[0][0]
print(test_url)

https://www.patriots.com/news/transcript-drake-maye-press-conference-1-29


In [25]:
# beautiful soup
request = requests.get(test_url, headers=HEADERS, timeout=20)
print("status", request.status_code)
print("some chars", request.text[:300])


status 200
some chars 

<!DOCTYPE html>
    <html lang="en-US" dir="ltr" />

<head>
    <meta charset="utf-8" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />

  <link rel="dns-prefetch" href="//nflenterprises.tt.omtrdc.ne


In [36]:
# extracting just the text from the test article / transcript
for tag in soup(["script","style","noscript","svg"]):
    tag.decompose()
article = soup.find("article")
if article is None:
    potentials = soup.find_all("div")
    article = max(potentials, key=lambda d: len(d.get_text(" ", strip=True)), default = soup)
text = article.get_text("\n", strip=True)
text = re.sub(r"\n{3,}","\n\n", text).strip()
print(text[:2000])


Primary nav
NEWS
VIDEO
PODCASTS
PHOTOS
SCHEDULE
TEAM
COMMUNITY
MOBILE APP
ALUMNI
CHEERLEADERS
TRAINING CAMP
HALL OF FAME
FAN ZONE
PRESS ROOM
PROSHOP
Search
Open menu button
Action related nav
TICKETS
PROSHOP
WATCH
LIVE AUDIO
LIVE NFL GAMES
Latest News
Analysis
Press Releases
Transcripts
Off the Field
NFL Draft
Training Camp
Auf Deutsch
Brasil
Español
Advertising
New England Patriots Press Conference and Conference Call Transcripts
TRANSCRIPTS
Transcripts: Head Coach Mike Vrabel, Quarterback Drake Maye and Wide Receiver Stefon Diggs Press Conferences 2/4
Read the full transcripts from press conferences from head coach Mike Vrabel, quarterback Drake Maye and wide receiver Stefon Diggs on Wednesday, February 4, 2026.
Transcript: Head Coach Mike Vrabel Press Conference 1/30
Read the full transcript from Head Coach Mike Vrabel's press conference on Friday, January 30, 2026.
Transcript: Head Coach Mike Vrabel Press Conference 1/29
Read the full transcript from Head Coach Mike Vrabel's press 

In [38]:
resp = requests.get(test_url, headers=HEADERS, timeout=20)
soup = BeautifulSoup(resp.text, "html.parser")
title_tag = soup.find("title")
print(title_tag.get_text(strip=True) if title_tag else "no title tag")

Transcript: Drake Maye Press Conference 1/29


In [40]:
body = soup.find("main") or soup.find("article") or soup
text = body.get_text("\n", strip=True)
print("some chars again:", len(text))
print(text[:1000])

some chars again: 16936
Official website of the New England Patriots
live
LIVE: 98.5 FM In-Game Radio Broadcast
Sun Feb 08 | 06:15 PM - 10:15 PM
UP NEXT
WHAT'S COMING UP NEXT:
Sun Feb 08 | 10:30 PM - 11:30 PM
live
LIVE: Super Bowl LX Postgame Press Conferences
Watch Super Bowl LX postgame press conferences immediately following the game.
VIEW MORE
Can't-Miss Play: Drake Maye's 35-yard TD launch to Hollins gets Pats on board in SB LX
Drake Maye's 24-yard dime to Hollins gets Pats to Seahawks' 35-yard line
Christian Gonzalez's third PBU of Super Bowl LX denies Cooper Kupp of catch
Milton Williams engulfs Sam Darnold for 8-yard sack in third quarter
Christian Gonzalez's full-extension goal-line PBU prevents Seahawks' third down scoring attempt
Drake Maye zips a 9-yard pass to Pop Douglas for a crucial third-down conversion
Can't-Miss Play: Christian Gonzalez goes airborne in full extension PBU on 52-yard pass to Shaheed
Woodson out-speeds Shaheed on 5-yard TFL chase down
Drake Maye's play

In [42]:
h1 = soup.find("h1")
headline = h1.get_text(" ",strip=True)
print("headline:", headline)

full_body = body.get_text("\n",strip=True)

start = full_body.find(headline)

clean_text = full_body[start:] if start != -1 else full_body

print(clean_text[:1000])

headline: Official website of the New England Patriots
Official website of the New England Patriots
live
LIVE: 98.5 FM In-Game Radio Broadcast
Sun Feb 08 | 06:15 PM - 10:15 PM
UP NEXT
WHAT'S COMING UP NEXT:
Sun Feb 08 | 10:30 PM - 11:30 PM
live
LIVE: Super Bowl LX Postgame Press Conferences
Watch Super Bowl LX postgame press conferences immediately following the game.
VIEW MORE
Can't-Miss Play: Drake Maye's 35-yard TD launch to Hollins gets Pats on board in SB LX
Drake Maye's 24-yard dime to Hollins gets Pats to Seahawks' 35-yard line
Christian Gonzalez's third PBU of Super Bowl LX denies Cooper Kupp of catch
Milton Williams engulfs Sam Darnold for 8-yard sack in third quarter
Christian Gonzalez's full-extension goal-line PBU prevents Seahawks' third down scoring attempt
Drake Maye zips a 9-yard pass to Pop Douglas for a crucial third-down conversion
Can't-Miss Play: Christian Gonzalez goes airborne in full extension PBU on 52-yard pass to Shaheed
Woodson out-speeds Shaheed on 5-yard T

In [44]:
for i, h in enumerate(soup.find_all("h1")):
    print(i,h.get_text(" ",strip=True))

0 Official website of the New England Patriots
1 Transcript: Drake Maye Press Conference 1/29


In [45]:
headline = soup.find_all("h1")[1].get_text(" ",strip=True)
print(headline)

full_body = body.get_text("\n",strip=True)

start = full_body.find(headline)
clean_text = full_body[start:] if start != -1 else full_body

print(clean_text[:1000])

Transcript: Drake Maye Press Conference 1/29
Transcript: Drake Maye Press Conference 1/29
Read the full transcript from Drake Maye's press conference on Thursday, January 29, 2026.
Jan 29, 2026 at 03:00 PM
New England Patriots
Copied!
PATRIOTS QUARTERBACK DRAKE MAYE
PRESS CONFERENCE
January 29, 2026
Q: How are you? How's your shoulder? It came up on the injury report, so everyone's curious how you're doing. How are you feeling?
DM: I'm feeling good. I got out there, moving around a good bit today. I was able to do some of the jog through, so I'm feeling good. I'm looking forward to being ready to go. This is the game you dream of playing in, so I'm looking forward to getting out there and getting a chance to play in the Super Bowl.
Q: Were you limited at all, or do you anticipate being limited moving forward?
DM: I'm just going to try to do whatever I can to get back to 100%. I think a lot of people in that locker room would say the same thing. Having a two-week break is nice to rest a

In [46]:
# parse the questions and answers to store the athlete's answers 

lines = [ln.strip() for ln in clean_text.splitlines() if ln.strip()]

turns = []
current = None

for ln in lines:
    match = re.match(r"^([A-Z]{1,4})\s*:\s*(.*)$",ln)
    if match:
        if current:
            current["text"] = current["text"].strip()
            turns.append(current)
        q = match.group(1)
        a = match.group(2)
        current = {"speaker": q, "text": a}
    else:
        if current:
            current["text"] += " " + ln
if current:
    current["text"] = current["text"].strip()
    turns.append(current)
print("turns:", len(turns))
print("first 6")
for t in turns[:6]:
    print(t["speaker"], t["text"][:120])

turns: 36
first 6
Q How are you? How's your shoulder? It came up on the injury report, so everyone's curious how you're doing. How are you f
DM I'm feeling good. I got out there, moving around a good bit today. I was able to do some of the jog through, so I'm feel
Q Were you limited at all, or do you anticipate being limited moving forward?
DM I'm just going to try to do whatever I can to get back to 100%. I think a lot of people in that locker room would say th
Q Do you think your ability to throw will be inhibited at all in the Super Bowl based on what you know about the injury so
DM No, I don't think so. That's why you spend time. That's why you have trainers. I think it's just been one of those thing


In [ ]:
# now that i have an idea of how that works, im going to scrape each team's 
# site for transcripts and then search those transcripts for the players on our list
# i will first take the players we have and make it a python list for searching



In [84]:
teams = [
    {"team":"Packers", "base":"https://www.packers.com"},
    {"team":"Steelers", "base":"https://www.steelers.com"},
    {"team":"Chiefs", "base":"https://www.chiefs.com"}
]
teams

[{'team': 'Packers', 'base': 'https://www.packers.com'},
 {'team': 'Steelers', 'base': 'https://www.steelers.com'},
 {'team': 'Chiefs', 'base': 'https://www.chiefs.com'}]

In [ ]:
# now that i know this is possible,
# going to work on automating it for the entire league
# Sitemaps for primary discovery mechanism, they list old URLs not just recent
# Also they are XML which is SICK

In [ ]:

# phase one: URL discovery 
# fetch a sitemap url, locate the <loc> tags for canonical url
# detect if it is a url or a sitemap index
    # recursively fetch the indexes for child sitemaps :/
    # if its a url, return all page urls

HEADERS = {"User-Agent": "CapstoneResearchBot/1.0"}
TIMOUT = 30
sitemap_url = "https://www.packers.com/sitemap.xml"

r = requests.get(sitemap_url, headers=HEADERS, timeout=TIMOUT)
r.raise_for_status()

xml_text = r.text

root = ET.fromstring(xml_text)

urls = [el.text for el in root.findall(".//{*}loc")]



transcripts_re = re.compile(
    r"(transcript|media-availability|media-availability|postgame|press[- ]conference|quotes)", 
    re.I
)
transcript_urls = [u for u in urls if transcripts_re.search(u)] 
print("urls in sitemap", len(urls))
for u in transcript_urls[:10]:
    print(u)


# not good enough

urls in sitemap 1064
https://www.packers.com/video/press-conferences


In [6]:
# gold mine
HEADERS = {"User-Agent": "CapstoneResearchBot/1.0"}

robots_url = "https://www.packers.com/robots.txt"
r = requests.get(robots_url, headers=HEADERS, timeout=30)
r.raise_for_status()

text = r.text
print(text[:1000])

sitemaps = re.findall(r"(?im)^s*Sitemap:|s*(\S+)", text)
print("sitemaps found:", len(sitemaps))
for s in sitemaps:
    print(s)
    

User-agent: *
Disallow: /search/
Disallow: /_libraries/
Disallow: /_mobileview/
Sitemap: https://www.packers.com/sitemap-index.xml
sitemaps found: 10
User-agent:
*
Disallow:
/search/
Disallow:
/_libraries/
Disallow:
/_mobileview/

https://www.packers.com/sitemap-index.xml


In [7]:
sitemap_index = "https://www.packers.com/sitemap-index.xml"

r = requests.get(sitemap_index, timeout=30)
r.raise_for_status()

root = ET.fromstring(r.text)

child_sitemaps = [el.text.strip() for el in root.findall(".//{*}loc")]

print("child sitemaps found:", len(child_sitemaps))
for s in child_sitemaps:
    print(s)
    

child sitemaps found: 11
https://www.packers.com/sitemap.xml
https://www.packers.com/sitemap-fast-changing.xml
https://www.packers.com/sitemap-video-fast-changing.xml
https://www.packers.com/sitemap-news-archive_1.xml
https://www.packers.com/sitemap-news-archive_2.xml
https://www.packers.com/sitemap-news-archive_3.xml
https://www.packers.com/sitemap-news-archive_4.xml
https://www.packers.com/sitemap-news-archive_5.xml
https://www.packers.com/sitemap-news-archive_6.xml
https://www.packers.com/sitemap-video-archive_1.xml
https://www.packers.com/sitemap-video-archive_2.xml


In [4]:
news_sitemap = "https://www.packers.com/sitemap-news-archive_1.xml"



r = requests.get(news_sitemap, timeout=30)
r.raise_for_status()

root = ET.fromstring(r.text)

child_sitemaps = [el.text.strip() for el in root.findall(".//{*}loc")]

print("urls found in archive 1:", len(child_sitemaps))
for s in child_sitemaps:
    print(s)

urls found in archive 1: 5000
https://www.packers.com/news/rapid-reaction-too-often-these-packers-made-life-too-difficult-on-themselves-wild-card-2025-season
https://www.packers.com/news/in-game-updates-nfc-wild-card-playoff-bears-2025-season
https://www.packers.com/news/mike-s-mid-week-chat-jan-7-2026
https://www.packers.com/news/mike-s-mid-week-chat-dec-31-2025
https://www.packers.com/news/game-notes-packers-explosive-plays-started-with-christian-watson-week-17-2025
https://www.packers.com/news/key-to-the-game-packers-fail-to-find-answer-for-derrick-henry-week-17-2025
https://www.packers.com/news/rapid-reaction-it-s-a-shame-packers-aren-t-celebrating-two-wins-thanks-to-malik-willis-week-17-2025
https://www.packers.com/news/packers-salute-u-s-army-veteran-adam-pierce-for-operation-fan-mail-week-17-2025
https://www.packers.com/news/packers-ravens-week-17-inactives-dec-27-2025
https://www.packers.com/news/packers-announce-roster-moves-dec-27-2025
https://www.packers.com/news/packers-vs-

In [7]:
robots_url = "https://www.patriots.com/robots.txt"
r = requests.get(robots_url, headers=HEADERS, timeout=30)
r.raise_for_status()

text = r.text
print(text[:1000])

sitemaps = re.findall(r"(?im)^s*Sitemap:|s*(\S+)", text)
print("sitemaps found:", len(sitemaps))
for s in sitemaps:
    print(s)

User-agent: *
Disallow: /search/
Disallow: /_libraries/
Disallow: /_mobileview/
Sitemap: https://www.patriots.com/sitemap-index.xml
sitemaps found: 10
User-agent:
*
Disallow:
/search/
Disallow:
/_libraries/
Disallow:
/_mobileview/

https://www.patriots.com/sitemap-index.xml


In [8]:
sitemap_index = "https://www.patriots.com/sitemap-index.xml"

r = requests.get(sitemap_index, timeout=30)
r.raise_for_status()

root = ET.fromstring(r.text)

child_sitemaps = [el.text.strip() for el in root.findall(".//{*}loc")]

print("child sitemaps found:", len(child_sitemaps))
for s in child_sitemaps:
    print(s)


child sitemaps found: 13
https://www.patriots.com/sitemap.xml
https://www.patriots.com/sitemap-fast-changing.xml
https://www.patriots.com/sitemap-video-fast-changing.xml
https://www.patriots.com/sitemap-news-archive_1.xml
https://www.patriots.com/sitemap-news-archive_2.xml
https://www.patriots.com/sitemap-news-archive_3.xml
https://www.patriots.com/sitemap-news-archive_4.xml
https://www.patriots.com/sitemap-news-archive_5.xml
https://www.patriots.com/sitemap-news-archive_6.xml
https://www.patriots.com/sitemap-news-archive_7.xml
https://www.patriots.com/sitemap-news-archive_8.xml
https://www.patriots.com/sitemap-video-archive_1.xml
https://www.patriots.com/sitemap-video-archive_2.xml


In [ ]:
# phase 1: automattion
    # for team in teams:
            # discover teame sitemap 
            # extract news archive URLS
            # save to CSV
# phase 2: player mentions detected
# phase 3: sentiment analysis / embeddings

In [ ]:
%pip install sentence_transformers
%pip install chromadb
%pip install --upgrade "pymongo"
%pip install feedparser newspaper3k nltk 
%pip install lxml_html_clean
%pip -q install pypdf tldextract
import requests
import os
import csv
import json
import pandas as pd
import feedparser
import chromadb
import nltk
nltk.download('punkt')
import nltk
nltk.download('punkt_tab')
import time
import re
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from chromadb import Client, Settings
from pymongo import MongoClient
from newspaper import Article
from nltk.tokenize import sent_tokenize
import hashlib
from xml.etree import ElementTree as ET
from urllib.parse import urljoin
from urllib.parse import urlparse

model = SentenceTransformer('sentence-transformers/LaBSE')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\vince\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
c:\Users\vince\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
# phase 1 comence
import json
with open("teams.json") as f:
    teams = json.load(f)

print(len(teams))


32


In [12]:
with open("teams.json","r",encoding="utf-8") as f:
    teams = json.load(f)
print("# of teams: " + str(len(teams)))
print(teams[:3])

# of teams: 32
[{'team': 'packers', 'base': 'https://www.packers.com'}, {'team': 'bears', 'base': 'https://www.chicagobears.com'}, {'team': 'lions', 'base': 'https://www.detroitlions.com'}]


In [13]:
HEADERS = {"User-Agent": "CapstoneResearchBot/1.0(project)"}

def get_sitemaps_from_robots(base_url: str) -> list[str]:
    robots_url = base_url.rstrip("/") + "/robots.txt"
    r = requests.get(robots_url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    sitemaps = []
    for line in r.text.splitlines():
        if line.lower().startswith("sitemap:"):
            sm = line.split(":", 1)[1].strip()
            if sm.startswith("http"):
                sitemaps.append(sm)
    # dedupe and preserve the order
    return list(dict.fromkeys(sitemaps))

In [14]:
# download a sitemap and returns either the sitemap index or urlset
def parse_sitemap(sitemap_url: str) -> tuple[str,list[str]]:
    r = requests.get(sitemap_url, headers=HEADERS, timeout=30)
    r.raise_for_status()

    root = ET.fromstring(r.text)
    tag = root.tag.lower()

    locs = [el.text.strip() for el in root.findall(".//{*}loc")]

    if tag.endswith("sitemapindex"):
        return "index", locs
    elif tag.endswith("urlset"):
        return "urlset", locs
    else:
        return "unknown", locs
    

In [15]:
# writing the crawler funtion that takes a list of the team sitemaps and a max number of sitemaps
# the crawler will scrape each sitemap XML structure for urls, 
# and if it finds another sitemap index, it will recursively scrape that as well until it reaches the max number of sitemaps to scrape
# the function will then return a list of the urls it found in the sitemap indexes, up to the max number of sitemaps to scrape
# let's go
def web_crawler(start_sitemaps: list[str], max_sitemaps: int = 300) -> list[str]:
    seen = set()
    queue = list(start_sitemaps)
    page_urls = []

    while queue and len(seen) < max_sitemaps:
        sm = queue.pop(0)
        if sm in seen:
            continue
        seen.add(sm)

        kind, locs = parse_sitemap(sm)
        if kind == "index":
            for child in locs:
                if child not in seen:
                    queue.append(child)
        elif kind == "urlset":
            page_urls.extend(locs)
    # dedupe page urls and preserve order
    return list(dict.fromkeys(page_urls))

In [16]:
def is_news_url(u:str) -> bool:
    return urlparse(u).path.startswith("/news/")
def save_urls(team: str, urls: list[str], out_dir = "discovered_urls") -> str:
    os.makedirs(out_dir, exist_ok = True)
    out_path = os.path.join(out_dir, f"{team}_news_urls.csv")

    with open(out_path, "w", newline = "", encoding= "utf-8") as f:
        w = csv.writer(f)
        w.writerow(["team", "url"])
        for u in urls:
            w.writerow([team, u])

    return out_path

In [19]:
import time
results = []

for t in teams:
    team = t["team"]
    base = t["base"]
    print("team: " + team)

    try:
        site_maps = get_sitemaps_from_robots(base)
        pages = web_crawler(site_maps)
        news_pages = [u for u in pages if is_news_url(u)]

        out_file = save_urls(team, news_pages)

        results.append((team, len(news_pages), out_file))
        print("found ", len(news_pages), "news pages" )

        print("saved " + out_file + " files ")

    except Exception as e:
        results.append((team, 0, "failed"))
        print("fails: ", type(e).__name__, str(e)[:120])
    time.sleep(1.5)

team: packers
found  26378 news pages
saved discovered_urls\packers_news_urls.csv files 
team: bears
found  10495 news pages
saved discovered_urls\bears_news_urls.csv files 
team: lions
found  10941 news pages
saved discovered_urls\lions_news_urls.csv files 
team: vikings
found  15638 news pages
saved discovered_urls\vikings_news_urls.csv files 
team: patriots
found  39272 news pages
saved discovered_urls\patriots_news_urls.csv files 
team: dolphins
found  4957 news pages
saved discovered_urls\dolphins_news_urls.csv files 
team: jets
found  17175 news pages
saved discovered_urls\jets_news_urls.csv files 
team: bills
found  14322 news pages
saved discovered_urls\bills_news_urls.csv files 
team: steelers
found  20670 news pages
saved discovered_urls\steelers_news_urls.csv files 
team: ravens
found  25642 news pages
saved discovered_urls\ravens_news_urls.csv files 
team: bengals
found  18534 news pages
saved discovered_urls\bengals_news_urls.csv files 
team: browns
found  15911 news pages

In [ ]:
# built a multi-site automated sitemap crawler 
# 500-700k total article URLS over time
# re usable struture for future projects

In [22]:
# i am going to take a slice of 1 team to make sure the urls are good

df = pd.read_csv("discovered_urls/packers_news_urls.csv")
urls = df["url"].tolist()

test_urls = urls[:50]

for u in test_urls:
    print(u)


https://www.packers.com/news/
https://www.packers.com/news/2023-50s-classic-uniforms
https://www.packers.com/news/5-things
https://www.packers.com/news/all-news
https://www.packers.com/news/cliff-christl-history
https://www.packers.com/news/cliffs-high-fives
https://www.packers.com/news/cliffs-notes
https://www.packers.com/news/cliffs-oral-history
https://www.packers.com/news/cliffs-profiling-the-past
https://www.packers.com/news/cliffs-research-blogs
https://www.packers.com/news/comment-policy
https://www.packers.com/news/countdown-to-camp
https://www.packers.com/news/dope-sheet
https://www.packers.com/news/draft-digest
https://www.packers.com/news/editorial
https://www.packers.com/news/espanol
https://www.packers.com/news/free-agency
https://www.packers.com/news/game-notes
https://www.packers.com/news/game-recap
https://www.packers.com/news/gameday-guide
https://www.packers.com/news/gameday-guide-away
https://www.packers.com/news/gameday-news
https://www.packers.com/news/infographics

In [27]:
# building an article filter to get rid of non-article pages
# longer titles are more news like and the shorter titles seem 
# to be more like video pages or something else we dont want

def is_likely_article(url: str) -> bool:
    if "/news/" not in url:
        return False
    slug = url.split("/news/")[-1]

    if "-" not in slug:
        return False
    
    if len(slug) <= 25:
        return False
    
    return True

In [28]:
article_urls = [u for u in urls if is_likely_article(u)]
print("likely article urls:", len(article_urls))
print(article_urls[:10])

likely article urls: 26157
['https://www.packers.com/news/dope-sheet-2025-season-review', 'https://www.packers.com/news/fans-reminded-of-project-play-60-set-for-feb-14-at-lambeau-field-2026', 'https://www.packers.com/news/packers-accepting-nominations-for-packers-protect-serve-award-feb-13-2026', 'https://www.packers.com/news/inbox-he-gave-it-a-shot-feb-13-2026', 'https://www.packers.com/news/packers-award-20-wisconsin-youth-football-programs-with-financial-assistance-grant-feb-12-2026', 'https://www.packers.com/news/titletown-crawl-offering-food-drink-and-entertainment-on-march-7-2026', 'https://www.packers.com/news/dependable-jordy-nelson-possessed-exceptional-feel-for-the-game-feb-12-2026', 'https://www.packers.com/news/inbox-that-unit-hits-the-reset-button-feb-12-2026', 'https://www.packers.com/news/inbox-there-s-always-an-element-of-luck-involved-feb-11-2026', 'https://www.packers.com/news/inbox-he-made-some-history-feb-10-2026']


In [29]:
# BeautifulSoup time
# download an article page
# extract the body text
# return it structured for document db storage.
# going to use session requests to be nice to the server and reuse connections

HEADERS = {
    "User-Agent": "CapstoneResearchBot/1.0(project)",
    "Accept-Language": "en-US,en;q=0.9",
    }
session = requests.Session()
session.headers.update(HEADERS)

In [33]:
# to avoid duplicates when storing the articles in the database, 
# I can use a hash of the url as the unique identifier for each article
import hashlib
def url_to_id(url: str) -> str:
    return hashlib.sha256(url.encode("utf-8")).hexdigest()


In [37]:
# fetch the article function
def fetch_article(url:str) -> dict:
    r = session.get(url, timeout=30)
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    # the text seems to be inside the <p> tags so i will start there

    paragraphs = [p.get_text(" ", strip=True) for p in soup.find_all("p")]

    body = " ".join(paragraphs)

    return{
        "id": url_to_id(url),
        "url": url,
        "body": body    
    }

In [41]:
# going to test the packers first
test_url = "https://www.packers.com/news/inbox-he-made-some-history-feb-10-2026"

article = fetch_article(test_url)

print(article["url"])
print(len(article["body"]))
print(article["body"][:1000])


https://www.packers.com/news/inbox-he-made-some-history-feb-10-2026
14014
packers.com editor Alan from Mount Auburn, IL Boring. That's all I got. It was not a game for the ages, but it was played, and I'm back. Jeff from Hagerstown, MD Just so you know, it will be nice when you don't have these four days off next year. No complaints. Jodi from Grand Rapids, WI They are never getting rid of the Roman numerals, are they? The opportunity to make the switch for good was 10 years ago, when they went with the numeral "50" instead of the L. But alas. Markus from Aurora, CO The Seahawks' SB LX victory may get in their fans' heads, but I don't mind Sam Darnold getting a ring. But it better be the Packers next year ... I can't imagine how Vikings fans felt these last few weeks seeing Darnold accomplish what he did. Kerry from Lakewood Ranch, FL Biggest D-line vs O-line mismatch in the Super Bowl sinceâ¦? Last year? The Seahawks' four-man rush controlled the game, just as Philly's did against KC

In [ ]:
sample_urls = article_urls[:20]

articles = []

for u in sample_urls:
    try:
        a = fetch_article(u)
        if a and "body" in a:
            articles.append(a)
            print("sick:", u)
        else:
            print("no dice: ", u)
    except Exception as e:
        print("error fetching ", u, type(e).__name__, str(e)[:120])
len(articles)
sum(len(a["body"]) for a in articles)
# The pipeline (crawl → filter → fetch → parse) is functioning !!!!

sick: https://www.packers.com/news/dope-sheet-2025-season-review
sick: https://www.packers.com/news/fans-reminded-of-project-play-60-set-for-feb-14-at-lambeau-field-2026
sick: https://www.packers.com/news/packers-accepting-nominations-for-packers-protect-serve-award-feb-13-2026
sick: https://www.packers.com/news/inbox-he-gave-it-a-shot-feb-13-2026
sick: https://www.packers.com/news/packers-award-20-wisconsin-youth-football-programs-with-financial-assistance-grant-feb-12-2026
sick: https://www.packers.com/news/titletown-crawl-offering-food-drink-and-entertainment-on-march-7-2026
sick: https://www.packers.com/news/dependable-jordy-nelson-possessed-exceptional-feel-for-the-game-feb-12-2026
sick: https://www.packers.com/news/inbox-that-unit-hits-the-reset-button-feb-12-2026
sick: https://www.packers.com/news/inbox-there-s-always-an-element-of-luck-involved-feb-11-2026
sick: https://www.packers.com/news/inbox-he-made-some-history-feb-10-2026
sick: https://www.packers.com/news/turning-the-pa

153627

In [59]:
#print("# articles collected: ", len(articles))

# i am going to store these articles in a JSONL file 
# for now, and then i will work on the embedding and 
# vector database storage next
# JSONL is a convenient format for storing 
# structured data where each line is a JSON object, 
# making it easy to read and write large datasets 
# without loading everything into memory at once, and 
# it is good for document and vector databases.

os.makedirs("raw_articles", exist_ok=True)
# create a folder named raw articles in the proj dicrectory
# write the above sample articles to a JSONL file in that folder 
# named packers_sample_articles.jsonl

out_path = "raw_articles/packers_articles.jsonl"

with open(out_path, "w", encoding="utf-8") as f:
    for a in articles:
        f.write(json.dumps(a, ensure_ascii=False) + "\n")
        # converts the python dict to a JSON string and adds a new line for the article
        # f.writes the line to the disk

print("wrote:",out_path)


wrote: raw_articles/packers_articles.jsonl


In [60]:
open(out_path, "w").close()
print("file reset:", out_path)

file reset: raw_articles/packers_articles.jsonl


In [ ]:
# now i am going to scale the previous code for all of the packers articles
os.makedirs("raw_articles", exist_ok=True)
out_path = "raw_articles/packers_sample.jsonl"
def append_jsonl(path:str, record: dict):
    with open(path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")
batch_size = 100
sleep_s = 1.0
# max_articles = 5000
total = 0
saved = 0

for i in range(0, len(article_urls), batch_size):
    batch = article_urls[i:i+batch_size]
    for url in batch:
        rec = fetch_article(url)
        if rec and rec.get("body") and len(rec["body"]) > 400:
            rec["team"] = "packers'"
            append_jsonl(out_path, rec)
            saved += 1
        # if saved >= max_articles:
            break
total += len(batch)
print(f"batch done. total: {total}, saved: {saved}")
time.sleep(sleep_s)


KeyboardInterrupt: 